In [1]:

import pandas as pd
import requests
import warnings
warnings.filterwarnings("ignore")

Ce notebook explique le processus d'extraction des molécules utilisées dans chaque essais


In [2]:
nct_id="NCT06317662"
val = f'https://clinicaltrials.gov/api/int/studies/{nct_id}?query.id={nct_id}&history=true'
response = requests.get(val)

if response.status_code == 200:
    # Parse the response as JSON
    data = response.json()

topic_drugInfo =  [item['name'] for item in data.get('topics', {}).get('drugInfo', [])]
if not topic_drugInfo:
                    topic_drugInfo = ['No drug info'] #default value
            
if 'study' in data:
    data_study = data['study']["protocolSection"]
#data_study

Premiere extraction des données de toutes les cohortes

Partie des infos concernant les cohortes

In [3]:
data_study['armsInterventionsModule']['armGroups'][0]

{'label': 'Arm A',
 'type': 'EXPERIMENTAL',
 'description': 'See Detailed Description for Arm A.',
 'interventionNames': ['Drug: Asparaginase Erwinia chrysanthemi',
  'Procedure: Biospecimen Collection',
  'Biological: Blinatumomab',
  'Procedure: Bone Marrow Aspiration',
  'Drug: Calaspargase Pegol',
  'Procedure: Computed Tomography',
  'Drug: Cyclophosphamide',
  'Drug: Cytarabine',
  'Drug: Daunorubicin',
  'Drug: Dexamethasone',
  'Procedure: Echocardiography',
  'Procedure: FDG-Positron Emission Tomography',
  'Drug: Leucovorin',
  'Procedure: Lumbar Puncture',
  'Procedure: Magnetic Resonance Imaging',
  'Drug: Mercaptopurine',
  'Drug: Methotrexate',
  'Procedure: Multigated Acquisition Scan',
  'Drug: Pegaspargase',
  'Drug: Therapeutic Hydrocortisone',
  'Drug: Vincristine']}

In [4]:
arm = data_study['armsInterventionsModule']['armGroups'][0]

print("Exctration du label/nom de la cohorte :")
label = arm.get('label', None)  # Default to None if not found
print(label)
print("\nExctration du type d'intervention exp/active comp/placebo :")
arm_type = arm.get('type', None)  # Default to None if not found
print(arm_type)
print("\nExctration de interventionNames:")
intervention_names = arm.get('interventionNames', [])  # Default to empty list if not found
print(intervention_names)
n=1
for intervention in intervention_names:
    print(f"\nExctraction de l'intervention {n} pour separer le nom du type de traitement:")
    try:
        # Split the intervention into type and name using ": " as the separator
        intervention_type, intervention_name = intervention.split(": ")
        print(f"Type: {intervention_type}")
        print(f"Name: {intervention_name}")
    except ValueError:
        # Handle cases where the intervention string does not contain ": "
        # Set intervention_type to None and use the full string as intervention_name
        intervention_type = None
        intervention_name = intervention  # Use the whole string if no split
    n+=1


Exctration du label/nom de la cohorte :
Arm A

Exctration du type d'intervention exp/active comp/placebo :
EXPERIMENTAL

Exctration de interventionNames:
['Drug: Asparaginase Erwinia chrysanthemi', 'Procedure: Biospecimen Collection', 'Biological: Blinatumomab', 'Procedure: Bone Marrow Aspiration', 'Drug: Calaspargase Pegol', 'Procedure: Computed Tomography', 'Drug: Cyclophosphamide', 'Drug: Cytarabine', 'Drug: Daunorubicin', 'Drug: Dexamethasone', 'Procedure: Echocardiography', 'Procedure: FDG-Positron Emission Tomography', 'Drug: Leucovorin', 'Procedure: Lumbar Puncture', 'Procedure: Magnetic Resonance Imaging', 'Drug: Mercaptopurine', 'Drug: Methotrexate', 'Procedure: Multigated Acquisition Scan', 'Drug: Pegaspargase', 'Drug: Therapeutic Hydrocortisone', 'Drug: Vincristine']

Exctraction de l'intervention 1 pour separer le nom du type de traitement:
Type: Drug
Name: Asparaginase Erwinia chrysanthemi

Exctraction de l'intervention 2 pour separer le nom du type de traitement:
Type: Pr

In [5]:
rows = []

for row in data_study:
    # Iterate over each arm group in the data
    for arm in data_study['armsInterventionsModule']['armGroups']:
        # Safely extract the 'label', 'type', and 'interventionNames' from each arm
        label = arm.get('label', None)  # Default to None if not found
        #print(label)
        arm_type = arm.get('type', None)  # Default to None if not found
        #print(arm_type)
        intervention_names = arm.get('interventionNames', [])  # Default to empty list if not found
        #print(intervention_names)
        
        for intervention in intervention_names:
                    try:
                        # Split the intervention into type and name using ": " as the separator
                        intervention_type, intervention_name = intervention.split(": ")
                    except ValueError:
                        # Handle cases where the intervention string does not contain ": "
                        # Set intervention_type to None and use the full string as intervention_name
                        intervention_type = None
                        intervention_name = intervention  # Use the whole string if no split
                    
                    rows.append({
                                    'label': label,
                                    'type': arm_type,
                                    'intervention': intervention_name,
                                    'intervention_type': intervention_type.upper() if intervention_type else None,  # Convert to uppercase if not None
                                    'otherNames': None  # Placeholder for other names
                                })
df = pd.DataFrame(rows)
df.head(10)


,label,type,intervention,intervention_type,otherNames
0,Arm A,EXPERIMENTAL,Asparaginase Erwinia chrysanthemi,DRUG,None
1,Arm A,EXPERIMENTAL,Biospecimen Collection,PROCEDURE,None
2,Arm A,EXPERIMENTAL,Blinatumomab,BIOLOGICAL,None
3,Arm A,EXPERIMENTAL,Bone Marrow Aspiration,PROCEDURE,None
4,Arm A,EXPERIMENTAL,Calaspargase Pegol,DRUG,None
5,Arm A,EXPERIMENTAL,Computed Tomography,PROCEDURE,None
6,Arm A,EXPERIMENTAL,Cyclophosphamide,DRUG,None
7,Arm A,EXPERIMENTAL,Cytarabine,DRUG,None
8,Arm A,EXPERIMENTAL,Daunorubicin,DRUG,None
9,Arm A,EXPERIMENTAL,Dexamethasone,DRUG,None


Partie concernant les molécules

In [6]:
data_study['armsInterventionsModule']['interventions'][0]

{'type': 'DRUG',
 'name': 'Asparaginase Erwinia chrysanthemi',
 'description': 'Given recombinant crisantaspase IM or crisantaspase IM or IV',
 'armGroupLabels': ['Arm A',
  'Arm B, Cohort 1',
  'Arm B, Cohort 2',
  'Arm B, Cohort 3',
  'Arm B, Cohort 4',
  'Safety Phase Cohort'],
 'otherNames': ['Asparaginase Erwinia chrysanthemi (Recombinant)-rywn',
  'Asparaginase Erwinia chrysanthemi, Recombinant-rywn',
  'Asparaginase Erwinia chrysanthemi-rywn',
  'Crisantaspase',
  'Crisantaspase Biobetter JZP-458',
  'Crisantaspasum',
  'Enrylaze',
  'Erwinase',
  'Erwinaze',
  'JZP 458',
  'JZP-458',
  'JZP458',
  'PF743',
  'RC-P JZP-458',
  'Recombinant Asparaginase erwinia chrysanthemi JZP-458',
  'Recombinant Crisantaspase JZP-458',
  'Recombinant Erwinia asparaginase JZP-458',
  'Rylaze']}

In [7]:
intervention=data_study['armsInterventionsModule']['interventions'][0]
print("Extraction du type de la molecule/intervention:")
intervention_type = intervention.get('type', None)  # Default to None if not found
print(intervention_type)
print("\nRecuperation de la liste des cohortes incluant cette molecule/intervention:")
arm_group_labels = intervention.get('armGroupLabels', [])  # Default to empty list if not found
print(f"Liste de {len(arm_group_labels)} cohortes")
print("\nRecuperation de la liste des synonymes/alt names de cette molecule/intervention:")
other_names = intervention.get('otherNames', None)  # Extract 'otherNames', default to None if not found
print(f"{len(other_names)} synonymes")

print("\nSi on a pu recuperer la liste des cohortes incluant cette molecule/intervention")
print("On va croisser cette information avec notre dataframe precedent où intervention['name'] correspond à df['intervention']")
print("même chose pour croisser les informations pour les synonymes")
if arm_group_labels:
    # Check each armGroupLabel and update the corresponding rows in the DataFrame
    for label in arm_group_labels:
        # Find the matching rows based on label and intervention name
        matching_rows = (df['label'] == label) & (df['intervention'] == intervention.get('name', None))
        
        # If there are matching rows, update the 'otherNames' column with a joined string of other names
        if matching_rows.any():  # Ensure that matching rows exist before updating
            df.loc[matching_rows, 'otherNames'] = ', '.join(other_names) if other_names else None

Extraction du type de la molecule/intervention:
DRUG

Recuperation de la liste des cohortes incluant cette molecule/intervention:
Liste de 6 cohortes

Recuperation de la liste des synonymes/alt names de cette molecule/intervention:
18 synonymes

Si on a pu recuperer la liste des cohortes incluant cette molecule/intervention
On va croisser cette information avec notre dataframe precedent où intervention['name'] correspond à df['intervention']
même chose pour croisser les informations pour les synonymes


In [8]:
for intervention in data_study['armsInterventionsModule']['interventions']:
        # Extract intervention type and arm group labels
        intervention_type = intervention.get('type', None)  # Default to None if not found
        arm_group_labels = intervention.get('armGroupLabels', [])  # Default to empty list if not found
        other_names = intervention.get('otherNames', None)  # Extract 'otherNames', default to None if not found
        
        if arm_group_labels:
            # Check each armGroupLabel and update the corresponding rows in the DataFrame
            for label in arm_group_labels:
                # Find the matching rows based on label and intervention name
                matching_rows = (df['label'] == label) & (df['intervention'] == intervention.get('name', None))
                
                # If there are matching rows, update the 'otherNames' column with a joined string of other names
                if matching_rows.any():  # Ensure that matching rows exist before updating
                    df.loc[matching_rows, 'otherNames'] = ', '.join(other_names) if other_names else None

# Remove the label column from the DataFrame
df1 = df.iloc[:, 1:]
df1.head(10)

,type,intervention,intervention_type,otherNames
0,EXPERIMENTAL,Asparaginase Erwinia chrysanthemi,DRUG,Asparaginase Erwinia chrysanthemi (Recombinant...
1,EXPERIMENTAL,Biospecimen Collection,PROCEDURE,"Biological Sample Collection, Biospecimen Coll..."
2,EXPERIMENTAL,Blinatumomab,BIOLOGICAL,"AMG 103, AMG-103, AMG103, Anti-CD19 x Anti-CD3..."
3,EXPERIMENTAL,Bone Marrow Aspiration,PROCEDURE,None
4,EXPERIMENTAL,Calaspargase Pegol,DRUG,"Asparaginase (Escherichia coli Isoenzyme II), ..."
5,EXPERIMENTAL,Computed Tomography,PROCEDURE,"CAT, CAT Scan, Computed Axial Tomography, Comp..."
6,EXPERIMENTAL,Cyclophosphamide,DRUG,"(-)-Cyclophosphamide, 2H-1,3,2-Oxazaphosphorin..."
7,EXPERIMENTAL,Cytarabine,DRUG,".beta.-Cytosine arabinoside, 1-.beta.-D-Arabin..."
8,EXPERIMENTAL,Daunorubicin,DRUG,"Daunomycin, Daunorrubicina, DNR, Leukaemomycin..."
9,EXPERIMENTAL,Dexamethasone,DRUG,"Aacidexam, Adexone, Aknichthol Dexa, Alba-Dex,..."


topic_drugInfo se trouve à la fin dans la partie derived, on la rajoute comme potentiels match au cas où l'ont n'a pas réussi à récuperer le nom de toutes les molecules/interventions

In [9]:
topic_drugInfo

['Blinatumomab', 'Venetoclax']

In [10]:
df2=df1.drop_duplicates()
df2['nct_id'] = nct_id
# apply the same value topic_drugInfo to all rows
df2['potential_match'] = [topic_drugInfo] * len(df2)
df2.head(5)

,type,intervention,intervention_type,otherNames,nct_id,potential_match
0,EXPERIMENTAL,Asparaginase Erwinia chrysanthemi,DRUG,Asparaginase Erwinia chrysanthemi (Recombinant...,NCT06317662,"[Blinatumomab, Venetoclax]"
1,EXPERIMENTAL,Biospecimen Collection,PROCEDURE,"Biological Sample Collection, Biospecimen Coll...",NCT06317662,"[Blinatumomab, Venetoclax]"
2,EXPERIMENTAL,Blinatumomab,BIOLOGICAL,"AMG 103, AMG-103, AMG103, Anti-CD19 x Anti-CD3...",NCT06317662,"[Blinatumomab, Venetoclax]"
3,EXPERIMENTAL,Bone Marrow Aspiration,PROCEDURE,None,NCT06317662,"[Blinatumomab, Venetoclax]"
4,EXPERIMENTAL,Calaspargase Pegol,DRUG,"Asparaginase (Escherichia coli Isoenzyme II), ...",NCT06317662,"[Blinatumomab, Venetoclax]"


Enfin on défini exp seul / exp vs actif / exp vs placebo avec la columne "type"

In [11]:
if "PLACEBO_COMPARATOR" in df2['intervention'] or "SHAM_COMPARATOR" in df2['intervention']:
    df2["type d'intervention"] = "experimental vs placebo"
elif "ACTIVE_COMPARATOR" in df2['intervention']:
    df2["type d'intervention"] = "experimental vs active"
else:
    df2["type d'intervention"] = "experimental seul"
df2

,type,intervention,intervention_type,otherNames,nct_id,potential_match,type d'intervention
0,EXPERIMENTAL,Asparaginase Erwinia chrysanthemi,DRUG,Asparaginase Erwinia chrysanthemi (Recombinant...,NCT06317662,"[Blinatumomab, Venetoclax]",experimental seul
1,EXPERIMENTAL,Biospecimen Collection,PROCEDURE,"Biological Sample Collection, Biospecimen Coll...",NCT06317662,"[Blinatumomab, Venetoclax]",experimental seul
2,EXPERIMENTAL,Blinatumomab,BIOLOGICAL,"AMG 103, AMG-103, AMG103, Anti-CD19 x Anti-CD3...",NCT06317662,"[Blinatumomab, Venetoclax]",experimental seul
3,EXPERIMENTAL,Bone Marrow Aspiration,PROCEDURE,None,NCT06317662,"[Blinatumomab, Venetoclax]",experimental seul
4,EXPERIMENTAL,Calaspargase Pegol,DRUG,"Asparaginase (Escherichia coli Isoenzyme II), ...",NCT06317662,"[Blinatumomab, Venetoclax]",experimental seul
5,EXPERIMENTAL,Computed Tomography,PROCEDURE,"CAT, CAT Scan, Computed Axial Tomography, Comp...",NCT06317662,"[Blinatumomab, Venetoclax]",experimental seul
6,EXPERIMENTAL,Cyclophosphamide,DRUG,"(-)-Cyclophosphamide, 2H-1,3,2-Oxazaphosphorin...",NCT06317662,"[Blinatumomab, Venetoclax]",experimental seul
7,EXPERIMENTAL,Cytarabine,DRUG,".beta.-Cytosine arabinoside, 1-.beta.-D-Arabin...",NCT06317662,"[Blinatumomab, Venetoclax]",experimental seul
8,EXPERIMENTAL,Daunorubicin,DRUG,"Daunomycin, Daunorrubicina, DNR, Leukaemomycin...",NCT06317662,"[Blinatumomab, Venetoclax]",experimental seul
9,EXPERIMENTAL,Dexamethasone,DRUG,"Aacidexam, Adexone, Aknichthol Dexa, Alba-Dex,...",NCT06317662,"[Blinatumomab, Venetoclax]",experimental seul
